# Window functions

In this notebook you will:
* solve analytical question using window functions

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, unix_timestamp, row_number, lead, avg
from pyspark.sql import Window

import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('WF I')
    .getOrCreate()
)

# Task 1

* compute avg time between two consecutive answers for each user that answered at least 2 questions

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

data_input_path = os.path.join(project_path, 'data/answers')

In [ ]:
answersDF = (
    spark
    .read
    .option('path', data_input_path)
    .load()
).cache()

<b>Take only users that answered at least 2 questions:</b>

Hint:
* Use count as a window function
* Create the window per user
* Filter only users with count > 1

In [ ]:
# for this window avoid using the sort, because if you sort the window
# you will have to add frame definition .rowsBetween(Window().unboundedPreceding, Window().unboundedFollowing)
# if you don't sort all records for a given user_id will be considered in the window
# it appears that with the sort the default frame is .rowsBetween(Window().unboundedPreceding, Window().currentRow)

w = Window().partitionBy('user_id')

data = (
    answersDF
    .filter(col('user_id').isNotNull())
    .withColumn('r', count('*').over(w))
    .filter(col('r') > 1)
)

In [ ]:
data.orderBy('user_id', 'answer_id').show(n=5)

<b>Compute the average time between answers:</b>

Hint:
* Define new window also per user but sorted by creation_date
* Use lead function to add new column which contains the next answer
* Compute the time difference (use unix_timestamp)
* Group by user and compute the average

In [ ]:
w2 = Window().partitionBy('user_id').orderBy('creation_date')

resultDF = (
    data
    .withColumn('next_answer', lead('creation_date').over(w2))
    .filter(col('next_answer').isNotNull())
    .withColumn('diff', unix_timestamp(col('next_answer')) - unix_timestamp(col('creation_date')))
    .select('user_id', 'r', 'diff')
    .groupBy('user_id')
    .agg(
        avg('diff').alias('avg_response_period')
    )
    .orderBy('avg_response_period')
)

In [ ]:
resultDF.show()

In [ ]:
spark.stop()